<a href="https://colab.research.google.com/github/altaki/Data-Science-Projects/blob/main/Code_Batoul_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.utils import resample

In [15]:
# Load the data from the Excel sheet
survey_data = pd.read_excel('Ba.xlsx')


In [16]:
# Check for NaN values in the dataset
nan_counts = survey_data.isna().sum()
print("NaN counts in the dataset:")
print(nan_counts)

# Handle NaN values by imputing with the mean (or other appropriate method)
if nan_counts.sum() > 0:
    survey_data = survey_data.fillna(survey_data.mean())

NaN counts in the dataset:
N°Obs                                                        0
2. 1. Avez-vous déjà vu une campagne de produit partage ?    0
4. 3. Avez-vous déjà acheté un produit partage ?             0
6. Les produits de Volvic me rassurent                       0
7. J’ai confiance dans la qualité des produits de Volvic     0
                                                            ..
Genre                                                        0
Situation_familiale                                          0
Niveau_etude                                                 0
Profession                                                   0
Revenu_net_mensuel                                           0
Length: 76, dtype: int64


In [17]:
# Define the demographic columns (ensure these names match exactly with your Excel sheet)
demographic_columns = ['Genre', 'Profession', 'Situation_familiale', 'Age', 'Niveau_etude']

# Identify the question columns by excluding demographic columns from the dataset
question_columns = [col for col in survey_data.columns if col not in demographic_columns]

# Separate the features (demographic + questions) and targets (questions)
X = survey_data[demographic_columns + question_columns]
y = survey_data[question_columns]

# Initialize and train a Random Forest regressor on all features
regressor = RandomForestRegressor()
regressor.fit(X, y)

# Number of new individuals to generate
new_individuals_count = 300

In [18]:

# Define quotas with numerical representations (corrected version)
quotas = {
    'Age': {1: 0.08, 2: 0.12, 3: 0.12, 4: 0.13, 5: 0.13, 6: 0.17},
    'Genre': {1: 0.48, 2: 0.52},  # 1: Male, 2: Female
    'Situation_familiale': {1: 0.4, 2: 0.4, 3: 0.2, 4: 0.0},
    'Niveau_etude': {1: 0.0, 2: 0.0, 3: 0.2, 4: 0.23, 5: 0.27, 6: 0.3},
    'Profession': {1: 0.18, 2: 0.12, 3: 0.13, 4: 0.0, 5: 0.15, 6: 0.25, 7: 0.0, 8: 0.12, 9: 0.05}
}

In [19]:
# Function to generate new individuals with specified quotas
def generate_new_individuals(quotas, total_count):
    new_data = {column: [] for column in quotas.keys()}

    for column, column_quotas in quotas.items():
        remaining_count = total_count
        category_counts = []

        for category, proportion in column_quotas.items():
            count = int(total_count * proportion)
            category_counts.append(count)
            remaining_count -= count

        # Adjust the last category to ensure the total count is exactly total_count
        if remaining_count != 0:
            category_counts[-1] += remaining_count

        for i, (category, _) in enumerate(column_quotas.items()):
            new_data[column].extend([category] * category_counts[i])

    # Shuffle the new data to ensure random distribution
    for column in new_data:
        np.random.shuffle(new_data[column])

    return pd.DataFrame(new_data)

In [20]:
# Function to sample with quotas
#def sample_with_quotas(data, quotas, total_count):
 #   samples = []
  #  for column, column_quotas in quotas.items():
   #     for category, proportion in column_quotas.items():
    #        count = int(total_count * proportion)
     #       category_samples = resample(data[data[column] == category], n_samples=count, replace=True)
      #      samples.append(category_samples)
    #result = pd.concat(samples)
    #return result

In [21]:

# Sample data with quotas for each demographic column
#new_individuals_demographics = sample_with_quotas(X[demographic_columns], quotas, new_individuals_count)

In [22]:
# Generate the new individuals based on quotas
new_individuals_demographics = generate_new_individuals(quotas, new_individuals_count)

In [23]:
new_individuals_demographics.shape

(300, 5)

In [34]:
new_individuals_demographics.tail()

,Age,Genre,Situation_familiale,Niveau_etude,Profession
295,6,1,1,4,5
296,4,2,3,6,6
297,5,1,1,5,5
298,6,2,2,4,5
299,2,1,3,5,6


In [27]:
# Create a DataFrame for new individuals with all columns, filling non-demographic columns with NaN
new_individuals = pd.DataFrame(index=np.arange(new_individuals_count), columns=survey_data.columns)
for column in quotas.keys():
    new_individuals[column] = new_individuals_demographics[column].values

# Combine the original data with the new individuals
combined_data = pd.concat([survey_data, new_individuals], ignore_index=True)

In [36]:
combined_data.tail()

,N°Obs,2. 1. Avez-vous déjà vu une campagne de produit partage ?,4. 3. Avez-vous déjà acheté un produit partage ?,6. Les produits de Volvic me rassurent,7. J’ai confiance dans la qualité des produits de Volvic,8. Acheter les produits de Volvic est une garantie de qualité,9. Volvic est sincère vis-à-vis de ses clients,10. Volvic est honnête vis-à-vis de ses clients,11. Volvic montre de l’intérêt pour ses clients,12. Je pense que Volvic renouvelle ses produits pour tenir compte des progrès de la recherche,...,69. _4. Je souhaite que le prélèvement destiné à l’association soit :,70. J’ai une préférence quand le produit partage est un produit de consommation courante (exemple : Pâtes Panzani vendus en faveur des Restaurants du Cœur),71. J’ai une préférence quand le produit partage est plutôt un produit de luxe (exemple : Bracelet vendu par Louis Vuitton en faveur de l’UNICEF),74. 8. Seriez-vous intéressé(e)s par le fait qu’une marque renouvelle sa campagne de produit partage ? (Par exemple 2 campagnes par an ou plus),Age,Genre,Situation_familiale,Niveau_etude,Profession,Revenu_net_mensuel
445,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,1,1,4,5,NaN
446,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4,2,3,6,6,NaN
447,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,5,1,1,5,5,NaN
448,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,6,2,2,4,5,NaN
449,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,2,1,3,5,6,NaN


In [29]:
#
# Ensure we have exactly the required number of new individuals
#while len(new_individuals_demographics) < new_individuals_count:
 #   additional_samples = sample_with_quotas(X[demographic_columns], quotas, new_individuals_count - len(new_individuals_demographics))
#    new_individuals_demographics = pd.concat([new_individuals_demographics, additional_samples])

# Remove duplicates and reset index
#new_individuals_demographics = new_individuals_demographics.drop_duplicates().reset_index(drop=True)

In [30]:
# If we have more rows than needed, truncate the DataFrame
#if len(new_individuals_demographics) > new_individuals_count:
 #   new_individuals_demographics = new_individuals_demographics.iloc[:new_individuals_count]

# Check the number of rows
#print(f"Number of rows in new_individuals_demographics: {len(new_individuals_demographics)}")

# Generate random answers for the survey questions
random_questions = np.random.randint(1, 6, size=(new_individuals_count, len(question_columns)))

# Create a DataFrame for new individuals with demographic data and random question responses
new_individuals = pd.concat([new_individuals_demographics.reset_index(drop=True), pd.DataFrame(random_questions, columns=question_columns)], axis=1)

# Ensure there are no NaN or infinite values in the new_individuals DataFrame
new_individuals.replace([np.inf, -np.inf], np.nan, inplace=True)
new_individuals.fillna(method='ffill', inplace=True)
new_individuals.fillna(method='bfill', inplace=True)

In [38]:
# Print out the new_individuals DataFrame to check for NaNs or invalid values
print(new_individuals.isnull().sum())
print(new_individuals)

# Predict final answers for the survey questions using the trained model
#new_individuals[question_columns] = regressor.predict(new_individuals[demographic_columns + question_columns])

predicted_answers = regressor.predict(new_individuals[demographic_columns + question_columns])
new_individuals[question_columns] = predicted_answers.astype(int)  # Convert predicted answers to integers


# Now, new_individuals DataFrame contains the generated demographic and question responses for 300 new individuals
print(new_individuals)

Age                                                                                                                                                            0
Genre                                                                                                                                                          0
Situation_familiale                                                                                                                                            0
Niveau_etude                                                                                                                                                   0
Profession                                                                                                                                                     0
                                                                                                                                                              ..
69. _4. Je souhaite que le prélève

In [39]:
new_individuals.head()

,Age,Genre,Situation_familiale,Niveau_etude,Profession,N°Obs,2. 1. Avez-vous déjà vu une campagne de produit partage ?,4. 3. Avez-vous déjà acheté un produit partage ?,6. Les produits de Volvic me rassurent,7. J’ai confiance dans la qualité des produits de Volvic,...,64. Je suis certain(e) que j'aimerai toujours Volvic qui soutient la cause de l’accès à l’eau de la même façon dans l’avenir,65. Je me sens très fidèle à Volvic qui soutient la cause de l’accès à l’eau,66. 1. Je souhaite que la prochaine opération de produit partage soit :,67. 2. Je souhaite que ce produit/service soit :,68. 3. Je souhaite que ce produit/service ait :,69. _4. Je souhaite que le prélèvement destiné à l’association soit :,70. J’ai une préférence quand le produit partage est un produit de consommation courante (exemple : Pâtes Panzani vendus en faveur des Restaurants du Cœur),71. J’ai une préférence quand le produit partage est plutôt un produit de luxe (exemple : Bracelet vendu par Louis Vuitton en faveur de l’UNICEF),74. 8. Seriez-vous intéressé(e)s par le fait qu’une marque renouvelle sa campagne de produit partage ? (Par exemple 2 campagnes par an ou plus),Revenu_net_mensuel
0,2,1,1,5,8,10,1,1,3,3,...,3,3,2,2,1,2,4,4,1,3
1,6,2,2,4,9,9,1,1,3,3,...,3,3,2,2,2,2,4,3,1,3
2,2,1,3,6,3,9,1,1,3,3,...,3,3,2,2,2,2,4,3,1,3
3,5,1,2,6,9,10,1,1,3,3,...,3,3,2,2,2,2,4,4,1,3
4,3,2,1,3,1,10,1,1,3,3,...,3,3,2,2,2,2,5,3,1,3


In [41]:
new_individuals.shape

(300, 76)

In [42]:
# Concatenate the two dataframes to create a dataframe with 450 rows
final_dataset = pd.concat([combined_data, new_individuals], ignore_index=True)

# Sort the dataframe by index
final_dataset.sort_index(inplace=True)

# Print out the final dataframe
print(final_dataset)

    N°Obs 2. 1. Avez-vous déjà vu une campagne de produit partage ?  \
0       1                                                  2          
1       2                                                  2          
2       3                                                  2          
3       4                                                  1          
4       5                                                  2          
..    ...                                                ...          
745     9                                                  1          
746     9                                                  1          
747    10                                                  1          
748     9                                                  1          
749    10                                                  1          

    4. 3. Avez-vous déjà acheté un produit partage ?  \
0                                                  2   
1                                  

In [43]:
# Save the final dataframe to an Excel file
final_dataset.to_excel('combined_data.xlsx', index=False)